# Imports & Installations

In [ ]:
%pip install -i https://pypi.gurobi.com gurobipy

In [ ]:
import gurobipy as gp
from gurobipy import GRB

In [ ]:
import numpy as np

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

# Helper Functions
######Please feel free to add / create any additional functions that might help

In [ ]:
def getTechPreferences(file):
  techfile = open(file, 'r')
  file_lines = techfile.readlines()

  # Creating the Preference List 
  pref = list(file_lines[0].strip("[").strip("]\n").split(", "))
  pref_list = []
  for i in pref:
    pref_list.append(int(i))
  
  # Mapping the Preference Pattern
  pattern = file_lines[1].strip("\n")

  file_lines.pop(0) # removing preference
  file_lines.pop(0) # removing the shift pattern
  file_lines.pop(0) # remove the "\n" line before the scores 
  
  #print("this is the first line now", file_lines[0])

  scores = []
  shifts = []
  # Accessing the Tech Scores
  for line in file_lines:
    if (int(file_lines.index(line)) % 2) ==0:
      scores.append(int(line))
    else:
      shifts.append(list(line.strip("[").strip("]\n").split(", ")))

  #Reformatting Shifts to Be Ints vs String
  shift_list = []
  for i in shifts:
    current_shift = []
    for j in i:
      current_shift.append(int(j))
    shift_list.append(current_shift)

  return pref_list, pattern, scores, shift_list

In [ ]:
def getDSMPrefs(file):
  dsmFile = open(file, "r")
  file_lines = dsmFile.readlines()

  open_hours = list(file_lines[0].strip("[").strip("]\n").split(", "))
  print("open hours:", open_hours)
  #open_hours_start = int(open_hours[0])
  #print("open_hours_start:", open_hours_start)
  #open_hours_end = int(open_hours[1])
  #print("open_hours_end:", open_hours_end)

  max_techs_working = int(file_lines[1])
  #print("max techs:", max_techs_working)
  min_techs_working = int(file_lines[2])
  #print("min techs:", min_techs_working)

  return open_hours, max_techs_working, min_techs_working

In [ ]:
def CreateDSMFile(filepath, openhours, min_techs, max_techs): #Need to improve this later to account for max_techs PER time index
  # Creating DSM File
  filewrite = filepath
  with open(filewrite, 'w') as f:
  #open_hours = [[TimeIndex('Monday 6:30AM'),TimeIndex('Saturday 3:00PM')],[TimeIndex('Sunday 6:30AM'),TimeIndex('Sunday 3:00PM')]]
  shop_hours_vector = 336*[0]
  for o in open_hours:
    for t in range(o[0],o[1]):
      shop_hours_vector[t] = 1 
  max_techs_working = min_tech
  min_techs_working = max_techs
  f.write(str(shop_hours_vector))
  f.write("\n")
  f.write(str(max_techs_working))
  f.write("\n")
  f.write(str(min_techs_working))


In [ ]:
def TechFIS(locNum, filepath, number_of_techs): 
  # returns the techfiles, tech information, and tech scores 
  techfiles = []
  score_values = []
  shift_vector_list = []
  for i in range(1,number_of_techs +1):
    #techfilename = '/content/drive/My Drive/PTL Capstone 2022-2023/Cleveland Tech Preferences/' + str(locNum)+ '_Tech'+ str(i)+ '.txt'
    techfilename = str(filepath) + str(locNum) + '_Tech'+ str(i)+ '.txt'
    techfiles.append(str(filename))

  for file in techfiles:  
    PL, pattern, scores, shift_vector = getTechPreferences(file)
    score_values.append(scores)
    shift_vector_list.append(shift_vector)

  return score_values, shift_vector_list

In [ ]:
def TimeIndex(datetime):
  ''' Takes in a string argument such as Monday 9AM and converts into time index [0-336] ''' 
  TI = 0 #default is 'Monday 12AM' 
  dt = datetime.lower()
  splitdt = dt.split()
  # DAY OF THE WEEK 
  if splitdt[0] == 'monday':
    TI += 0
  if splitdt[0] == 'tuesday':
    TI += 48
  if splitdt[0] == 'wednesday':
    TI += 48*2
  if splitdt[0] == 'thursday':
    TI += 48*3
  if splitdt[0] == 'friday':
    TI += 48*4
  if splitdt[0] == 'saturday':
    TI += 48*5
  if splitdt[0] == 'sunday':
    TI += 48*6
  # AM OR PM
  if splitdt[1][-2:] == 'pm':
    TI += 24
  # TIME
  if splitdt[1][-4] == '3':#Half an hour is one index of time
    TI += 1
  if splitdt[1][-8:-5] != '12': # Midnight 
    TI += int(splitdt[1][-8:-5])*2
  return TI

# Shift Vector Generator

# Scoring Function

In [ ]:
# COMPARE PREFERENCES WITH SHIFTS - SUPER BASIC DRAFT

def calculate_mismatch(tech_pref, shift_pattern, open_close):
    mismatch = 0
    for i in range(len(tech_pref)):
      if int(shift_pattern[i]) == 1:
        if int(open_close[i])==0:
          mismatch += 100
        if int(tech_pref[i]) != int(shift_pattern[i]):
            mismatch += 1
    return mismatch
    # Score is the number of mismatched 

In [ ]:
def tech_score(tech_pref_file, DSM_file):
  
  # Reading Technician Preference File
  reading_tech_pref = open(tech_pref_file, 'r')
  techfile_line = reading_tech_pref.readlines()
  
  # Reformating the Tech Preferences 
  tech_pref1 = techfile_line[0]
  tech_pref = list(tech_pref1.strip("[").strip("]\n").split(", "))

  # Reading DSM File
  reading_dsm_file = open(DSM_file, 'r')
  DSMfile_line = reading_dsm_file.readlines()

  # Reformatting Open/Close Pattern
  open_close1 = DSMfile_line[0]
  open_close = list(open_close1.strip("[").strip("]\n").split(", "))
  print("open_close1", open_close1)
  
  var = techfile_line[1]
  # Reading the Preferred Shift Type
  if techfile_line[1] == '5x8':
    shift_pattern_choosen ='/content/drive/My Drive/PTL Capstone 2022-2023/Schedule Vectors/5_8_SCHEDULE_VECTOR.txt'
  if techfile_line[1] == '4x10':
    shift_pattern_choosen ='/content/drive/My Drive/PTL Capstone 2022-2023/Schedule Vectors/4_10_SCHEDULE_VECTOR.txt'
  if techfile_line[1] == '3x12':
    shift_pattern_choosen = '/content/drive/My Drive/PTL Capstone 2022-2023/Schedule Vectors/3_12_SCHEDULE_VECTOR.txt'

  # Looking at Shift Pattern 
  shift_pattern_choosen ='/content/drive/My Drive/PTL Capstone 2022-2023/Schedule Vectors/5_8_SCHEDULE_VECTOR.txt'
  reading_files = open(shift_pattern_choosen, 'r')
  file_line_list = reading_files.readlines()

  with open(tech_pref_file, 'w') as f:
    f.write(techfile_line[0])
    f.write(techfile_line[1])
    f.write("\n")
    for shift in file_line_list:
      shift_reformated = list(shift.strip("[").strip("]\n").split(", "))
      current_score = calculate_mismatch(tech_pref, shift_reformated, open_close)
      print("current score", current_close)
      if current_score <=25:
        f.write(str(current_score))
        f.write("\n")
        f.write(str(shift))
        #f.write("\n")

# Schedule Optimzation 

In [ ]:
def generateSchedule(locNum, number_of_techs, dsmFilepath, techfilepath ):

  score_values, shift_vector_list = TechFIS(locNum, techfilepath, number_of_techs)

  dsmFile = dsmFilePath
  open_hours_vector, max_techs_working, min_techs_working = getDSMPrefs(dsmFile)

############ Define Model ############
  model = gp.Model()
############ Decision Variables ############
  x= model.addVars([(tech, shift) for tech in range(1, number_of_techs+1) for shift in range(0, len(score_values[tech-1]))], vtype = GRB.BINARY, name = 'x')
  
  ############ Objective Function #############
  obj = gp.quicksum(x[tech,shift]*score_values[tech-1][shift] for tech in range(1,number_of_techs+1) for shift in range(0, len(score_values[tech-1])))
  
  # Set the objective to a minimize
  model.setObjective(obj, GRB.MINIMIZE)

  ############ Constraints ##############

  # Each Tech Must be Assigned a Shift
  for tech in range(1, number_of_techs+1):
    model.addConstr(gp.quicksum(x[tech,shift] for shift in range(0, len(score_values[tech-1]))) ==1)

  # There must be a minimum of 2 techs working while the shop is open
  for time in range(len(open_hours_vector)):
    if open_hours_vector[time] == '1':
      model.addConstr(gp.quicksum(x[tech,shift]*shift_vector_list[tech-1][shift][time] for tech in range(1,number_of_techs+1) for shift in range(0,len(score_values[tech-1]))) >= min_techs_working)
      #Create a Constraint for Max number of techs
      model.addConstr(gp.quicksum(x[tech,shift]*shift_vector_list[tech-1][shift][time] for tech in range(1,number_of_techs+1) for shift in range(0,len(score_values[tech-1]))) <= max_techs_working)
    
  # Model is now completed and ready to optimzie
  model.update()
  model.optimize()